# Get acknowledgements from digitised journals

The landing page of a digitised journal in Trove retrieves any acknowledgements using an internal API. In this notebook I've used this API to get acknowledgements from the [list of digital journals with OCRd text](digital-journals-with-text.csv) that I [created here](Download-text-for-all-digitised-journals.ipynb).

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook
from IPython.display import display, HTML

In [11]:
url = 'http://nla.gov.au/{}/details/ack'
df = pd.read_csv('digital-journals-with-text.csv', keep_default_na=False)

supporters = []
for row in tqdm_notebook(df.itertuples()):
    ack_url = url.format(row.trove_id)
    response = requests.get(ack_url)
    soup = BeautifulSoup(response.text)
    try:
        acks = soup.find(class_='list-acknowledgements').find_all('strong')
    except AttributeError:
        pass
    else:
        for ack in acks:
            supporters.append({'trove_id': row.trove_id, 'title': row.title, 'supporter': ack.string})

dfs = pd.DataFrame(supporters)

In [12]:
dfs.head()

,supporter,title,trove_id
0,NSW Soil Conservation Service,"Journal of soil conservation, New South Wales",nla.obj-740911077
1,Chris Edye,The New South Wales school magazine of literat...,nla.obj-748141557
2,NSW Dept. of Education,The New South Wales school magazine of literat...,nla.obj-748141557
3,The School Magazine,The New South Wales school magazine of literat...,nla.obj-748141557
4,State Library of NSW Digital Excellence Program,The New South Wales Aborigines' advocate : a m...,nla.obj-554406117


## Total acknowledgements

In [53]:
dfs['supporter'].value_counts().to_frame()

,supporter
State Library of NSW Digital Excellence Program,47
State Library of Victoria,4
UNSW Australia,4
Victorian Canine Association's Laurie Luxmoore Library,3
Chris Edye,3
State Library of New South Wales,3
Public Record Office Victoria,3
State Library of Queensland,3
Victoria State Government,3
Northern Peninsula Area State College,3


## Acknowledgements by journal

In [51]:
titles = dfs.groupby('title')
html = ''
for title, supporters in titles:
    html += '<h4>{}</h4><ul>'.format(title)
    for supporter in supporters.itertuples():
        html += '<li>{}</li>'.format(supporter.supporter)  
    html += '</ul>'
display(HTML(html))